In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install nltk

**Codice che calcola la similarità, adegua la lunghezza dei testi, da come risultato file excel**

In [ ]:
import os
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize
import glob
import re
import nltk
import random

# Scarica i dati di punteggiatura di nltk se non li hai già
nltk.download('punkt')

# Directory principale
main_directory = "/content/drive/MyDrive/Wiki download/Puliti_GastronomiaE_en_it_es_de_fr"

# Trova tutte le sottocartelle all'interno della cartella principale
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Carica il tokenizer e il modello di LABSE
tokenizer_labse = BertTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = BertModel.from_pretrained("sentence-transformers/LaBSE")

# Funzione per ottenere l'embedding di una frase utilizzando LABSE
def get_labse_embedding(sentences):
    tokens = tokenizer_labse(sentences, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Itera su tutte le sottocartelle
for subdirectory in subdirectories:
    subdirectory_path = os.path.join(main_directory, subdirectory)
    # Trova tutti i file di testo nella sottocartella corrente
    file_paths = glob.glob(os.path.join(subdirectory_path, '*.txt'))

    # Verifica se la sottocartella corrente contiene file di testo
    if file_paths:
        # Esegue il codice per questa sottocartella

        languages = []
        data = {}

        # Carica i dati dai file di testo e suddividi le frasi
        for file_path in file_paths:
            lang = re.search(r'_([a-zA-Z]{2})\.txt$', file_path).group(1).upper()
            languages.append(lang)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                sentences = sent_tokenize(text)
                data[lang] = sentences

        # Trova la lunghezza massima tra tutti i dati
        max_length = max(len(data[lang]) for lang in languages)

        # Riempire o tagliare i dati in modo che abbiano la stessa lunghezza per tutte le lingue
        for lang in languages:
            while len(data[lang]) < max_length:
                # Se il numero di frasi è inferiore alla lunghezza massima, aggiungi casualmente una frase dai dati esistenti
                data[lang].append(random.choice(data[lang]))
            if len(data[lang]) > max_length:
                # Se il numero di frasi è superiore alla lunghezza massima, taglia i dati
                data[lang] = data[lang][:max_length]

        # Calcola l'embedding per le frasi nelle diverse lingue
        embeddings = {lang: get_labse_embedding(data[lang]) for lang in languages}

        # Calcola la similarità coseno tra i vettori di embedding per le diverse coppie di lingue
        cosine_similarities = {}
        for i, lang1 in enumerate(languages):
            for j, lang2 in enumerate(languages):
                if i < j:  # Per evitare duplicati, calcoliamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                    key = f"Cosine_{lang1}_{lang2}"
                    cosine_similarities[key] = [cosine_similarity([emb1], [emb2])[0][0] for emb1, emb2 in zip(embeddings[lang1], embeddings[lang2])]

        # Costruisci un DataFrame per i risultati
        df_pairs = pd.DataFrame(columns=["LANGUAGE1", "SENTENCE1", "LANGUAGE2", "SENTENCE2", "COSINE SIMILARITY"])

        # Iterate over the languages to extract unique sentence pairs and cosine similarities
        for i, lang1 in enumerate(languages):
            for j, lang2 in enumerate(languages):
                if i < j:  # Per evitare duplicati, consideriamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                    key = f"Cosine_{lang1}_{lang2}"
                    if key in cosine_similarities:
                        # Extract unique sentence pairs and cosine similarities for the language pair
                        sentences1 = data[lang1]
                        sentences2 = data[lang2]
                        similarities = cosine_similarities[key]

                        # Create a DataFrame for the language pair
                        df_lang_pair = pd.DataFrame({
                            "LANGUAGE1": [lang1] * len(sentences1),
                            "SENTENCE1": sentences1,
                            "LANGUAGE2": [lang2] * len(sentences2),
                            "SENTENCE2": sentences2,
                            "COSINE SIMILARITY": similarities
                        })

                        # Append the DataFrame to df_pairs
                        df_pairs = pd.concat([df_pairs, df_lang_pair], ignore_index=True)

        # Write the DataFrame to an Excel file
        output_excel_path = f"/content/drive/MyDrive/{subdirectory}_similarity.xlsx"
        df_pairs.to_excel(output_excel_path, index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Codice per controllare le frasi per ogni lingua, in modo tale che abbiano tutte la stessa lunghezza**

In [ ]:
import os
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from nltk.tokenize import PunktSentenceTokenizer
import glob
import re
import nltk
import random

# Scarica i dati di punteggiatura di nltk se non li hai già
nltk.download('punkt')

# Directory principale
main_directory = "/content/drive/MyDrive/Wiki download/Puliti_GastronomiaE_en_it_es_de_fr"

# Trova tutte le sottocartelle all'interno della cartella principale
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Carica il tokenizer e il modello di LABSE
tokenizer_labse = BertTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = BertModel.from_pretrained("sentence-transformers/LaBSE")

# Funzione per ottenere l'embedding di una frase utilizzando LABSE
def get_labse_embedding(sentences):
    tokens = tokenizer_labse(sentences, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Itera su tutte le sottocartelle
for subdirectory in subdirectories:
    subdirectory_path = os.path.join(main_directory, subdirectory)
    # Trova tutti i file di testo nella sottocartella corrente
    file_paths = glob.glob(os.path.join(subdirectory_path, '*.txt'))

    # Verifica se la sottocartella corrente contiene file di testo
    if file_paths:
        # Esegue il codice per questa sottocartella

        # Function to extract language code from the filename
        def extract_language_code(filename):
            return re.search(r'_([a-zA-Z]{2})\.txt$', filename).group(1).upper()

        languages = []

        data = {}

        # Carica i dati dai file di testo e suddividi le frasi
        for file_path in file_paths:
            lang = extract_language_code(os.path.basename(file_path))
            languages.append(lang)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                # Creazione del tokenizer di frasi
                tokenizer_sentence = PunktSentenceTokenizer()
                # Divisione del testo in frasi
                sentences = tokenizer_sentence.tokenize(text)
                data[lang] = sentences

        # Trova la lunghezza massima tra tutti i dati
        max_length = max(len(data[lang]) for lang in languages)

        # Riempire o tagliare i dati in modo che abbiano la stessa lunghezza per tutte le lingue
        for lang in languages:
          max_lang_length = len(max(data[lang], key=len)) # Trova la lunghezza massima per la lingua corrente
          while len(data[lang]) < max_lang_length:
               # Aggiungi casualmente una frase dai dati esistenti fino a raggiungere la lunghezza massima per quella lingua
                data[lang].append(random.choice(data[lang]))

        # Stampa il numero di frasi per ogni lingua dopo il processo di riempimento o taglio dei dati
        print(f"Numero di frasi dopo il processo per '{subdirectory}':")
        for lang in languages:
            print(f"{lang}: {len(data[lang])}")

        # Calcola l'embedding per le frasi nelle diverse lingue
        embeddings = {lang: get_labse_embedding(data[lang]) for lang in languages}

        # Calcola la similarità coseno tra i vettori di embedding per le diverse coppie di lingue
        cosine_similarities = {}
        for i, lang1 in enumerate(languages):
            for j, lang2 in enumerate(languages):
                if i < j:  # Per evitare duplicati, calcoliamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                    key = f"Cosine_{lang1}_{lang2}"
                    cosine_similarities[key] = [cosine_similarity([emb1], [emb2])[0][0] for emb1, emb2 in zip(embeddings[lang1], embeddings[lang2])]

        # Costruisci un DataFrame per i risultati
        df_cosine = pd.DataFrame(cosine_similarities)

        # Create an empty DataFrame to store sentence pairs and cosine similarities
        df_pairs = pd.DataFrame(columns=["LANGUAGE1", "SENTENCE1", "LANGUAGE2", "SENTENCE2", "COSINE SIMILARITY"])

        # Iterate over the languages to extract unique sentence pairs and cosine similarities
        for i, lang1 in enumerate(languages):
            for j, lang2 in enumerate(languages):
                if i < j:  # Per evitare duplicati, consideriamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                    key = f"Cosine_{lang1}_{lang2}"
                    if key in cosine_similarities:
                        # Extract unique sentence pairs and cosine similarities for the language pair
                        sentences1 = data[lang1]
                        sentences2 = data[lang2]
                        similarities = cosine_similarities[key]

                        # Create a DataFrame for the language pair
                        df_lang_pair = pd.DataFrame({
                            "LANGUAGE1": [lang1] * len(sentences1),
                            "SENTENCE1": sentences1,
                            "LANGUAGE2": [lang2] * len(sentences2),
                            "SENTENCE2": sentences2,
                            "COSINE SIMILARITY": similarities
                        })

                        # Append the DataFrame to df_pairs
                        df_pairs = pd.concat([df_pairs, df_lang_pair], ignore_index=True)

        # Write the DataFrame to an Excel file
        output_excel_path = f"/content/drive/MyDrive/{subdirectory}_similarity.xlsx"
        df_pairs.to_excel(output_excel_path, index=False)

        print(data["FR"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Numero di frasi dopo il processo per 'Shepherds Pie_en-it-es-de-fr':
ES: 262
IT: 354
FR: 1095
DE: 201
EN: 361


**Codice che fa esattamente tutto quello che fa il precedente, ma cerca di sfruttare la parallelizzazione**

In [ ]:
import os
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize
import glob
import re
import nltk
import random
from concurrent.futures import ProcessPoolExecutor

# Scarica i dati di punteggiatura di nltk se non li hai già
nltk.download('punkt')

# Directory principale
main_directory = "/content/drive/MyDrive/Wiki download/Puliti_GastronomiaE_en_it_es_de_fr"

# Trova tutte le sottocartelle all'interno della cartella principale
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Carica il tokenizer e il modello di LABSE
tokenizer_labse = BertTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = BertModel.from_pretrained("sentence-transformers/LaBSE")

# Funzione per ottenere l'embedding di una frase utilizzando LABSE
def get_labse_embedding(sentences):
    tokens = tokenizer_labse(sentences, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Funzione per calcolare le similarità coseno tra coppie di lingue
def calculate_cosine_similarity(lang1, lang2, embeddings):
    similarities = [cosine_similarity([emb1], [emb2])[0][0] for emb1, emb2 in zip(embeddings[lang1], embeddings[lang2])]
    return similarities

# Itera su tutte le sottocartelle
for subdirectory in subdirectories:
    subdirectory_path = os.path.join(main_directory, subdirectory)
    # Trova tutti i file di testo nella sottocartella corrente
    file_paths = glob.glob(os.path.join(subdirectory_path, '*.txt'))

    # Verifica se la sottocartella corrente contiene file di testo
    if file_paths:
        # Esegue il codice per questa sottocartella

        languages = []
        data = {}

        # Carica i dati dai file di testo e suddividi le frasi
        for file_path in file_paths:
            lang = re.search(r'_([a-zA-Z]{2})\.txt$', file_path).group(1).upper()
            languages.append(lang)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                sentences = sent_tokenize(text)
                data[lang] = sentences

        # Trova la lunghezza massima tra tutti i dati
        max_length = max(len(data[lang]) for lang in languages)

        # Riempire o tagliare i dati in modo che abbiano la stessa lunghezza per tutte le lingue
        for lang in languages:
            while len(data[lang]) < max_length:
                # Se il numero di frasi è inferiore alla lunghezza massima, aggiungi casualmente una frase dai dati esistenti
                data[lang].append(random.choice(data[lang]))
            if len(data[lang]) > max_length:
                # Se il numero di frasi è superiore alla lunghezza massima, taglia i dati
                data[lang] = data[lang][:max_length]

        # Calcola l'embedding per le frasi nelle diverse lingue
        embeddings = {lang: get_labse_embedding(data[lang]) for lang in languages}

        # Calcola le similarità coseno tra le coppie di lingue in parallelo
        with ProcessPoolExecutor() as executor:
            futures = []
            for i, lang1 in enumerate(languages):
                for j, lang2 in enumerate(languages):
                    if i < j:  # Per evitare duplicati, calcoliamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                        futures.append(executor.submit(calculate_cosine_similarity, lang1, lang2, embeddings))

            # Costruisci un DataFrame per i risultati
            df_pairs = pd.DataFrame(columns=["LANGUAGE1", "SENTENCE1", "LANGUAGE2", "SENTENCE2", "COSINE SIMILARITY"])

            # Iterate over the languages to extract unique sentence pairs and cosine similarities
            index = 0
            for i, lang1 in enumerate(languages):
                for j, lang2 in enumerate(languages):
                    if i < j:  # Per evitare duplicati, consideriamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                        similarities = futures[index].result()
                        key = f"Cosine_{lang1}_{lang2}"
                        # Extract unique sentence pairs and cosine similarities for the language pair
                        sentences1 = data[lang1]
                        sentences2 = data[lang2]

                        # Create a DataFrame for the language pair
                        df_lang_pair = pd.DataFrame({
                            "LANGUAGE1": [lang1] * len(sentences1),
                            "SENTENCE1": sentences1,
                            "LANGUAGE2": [lang2] * len(sentences2),
                            "SENTENCE2": sentences2,
                            "COSINE SIMILARITY": similarities
                        })

                        # Append the DataFrame to df_pairs
                        df_pairs = pd.concat([df_pairs, df_lang_pair], ignore_index=True)
                        index += 1

        # Write the DataFrame to an Excel file
        output_excel_path = f"/content/drive/MyDrive/{subdirectory}_similarity.xlsx"
        df_pairs.to_excel(output_excel_path, index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os

**Codice per controllare i tempi di esecuzione e possibili bottleneck nel codice**

In [ ]:
import os
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
from nltk.tokenize import sent_tokenize
import glob
import re
import nltk
import random
from concurrent.futures import ProcessPoolExecutor
import cProfile

# Scarica i dati di punteggiatura di nltk se non li hai già
nltk.download('punkt')

# Directory principale
main_directory = "/content/drive/MyDrive/Wiki download/Puliti_GastronomiaE_en_it_es_de_fr"

# Trova tutte le sottocartelle all'interno della cartella principale
subdirectories = [subdir for subdir in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, subdir))]

# Carica il tokenizer e il modello di LABSE
tokenizer_labse = BertTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = BertModel.from_pretrained("sentence-transformers/LaBSE")

# Funzione per ottenere l'embedding di una frase utilizzando LABSE
def get_labse_embedding(sentences):
    tokens = tokenizer_labse(sentences, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Funzione per calcolare le similarità coseno tra coppie di lingue
def calculate_cosine_similarity(lang1, lang2, embeddings):
    similarities = [cosine_similarity([emb1], [emb2])[0][0] for emb1, emb2 in zip(embeddings[lang1], embeddings[lang2])]
    return similarities

def main():
    # Itera su tutte le sottocartelle
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(main_directory, subdirectory)
        # Trova tutti i file di testo nella sottocartella corrente
        file_paths = glob.glob(os.path.join(subdirectory_path, '*.txt'))

        # Verifica se la sottocartella corrente contiene file di testo
        if file_paths:
            # Esegue il codice per questa sottocartella

            languages = []
            data = {}

            # Carica i dati dai file di testo e suddividi le frasi
            for file_path in file_paths:
                lang = re.search(r'_([a-zA-Z]{2})\.txt$', file_path).group(1).upper()
                languages.append(lang)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    sentences = sent_tokenize(text)
                    data[lang] = sentences

            # Trova la lunghezza massima tra tutti i dati
            max_length = max(len(data[lang]) for lang in languages)

            # Riempire o tagliare i dati in modo che abbiano la stessa lunghezza per tutte le lingue
            for lang in languages:
                while len(data[lang]) < max_length:
                    # Se il numero di frasi è inferiore alla lunghezza massima, aggiungi casualmente una frase dai dati esistenti
                    data[lang].append(random.choice(data[lang]))
                if len(data[lang]) > max_length:
                    # Se il numero di frasi è superiore alla lunghezza massima, taglia i dati
                    data[lang] = data[lang][:max_length]

            # Calcola l'embedding per le frasi nelle diverse lingue
            embeddings = {lang: get_labse_embedding(data[lang]) for lang in languages}

            # Calcola le similarità coseno tra le coppie di lingue in parallelo
            with ProcessPoolExecutor() as executor:
                futures = []
                for i, lang1 in enumerate(languages):
                    for j, lang2 in enumerate(languages):
                        if i < j:  # Per evitare duplicati, calcoliamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                            futures.append(executor.submit(calculate_cosine_similarity, lang1, lang2, embeddings))

                # Costruisci un DataFrame per i risultati
                df_pairs = pd.DataFrame(columns=["LANGUAGE1", "SENTENCE1", "LANGUAGE2", "SENTENCE2", "COSINE SIMILARITY"])

                # Iterate over the languages to extract unique sentence pairs and cosine similarities
                index = 0
                for i, lang1 in enumerate(languages):
                    for j, lang2 in enumerate(languages):
                        if i < j:  # Per evitare duplicati, consideriamo solo una volta per ciascuna coppia (lang1, lang2) dove lang1 < lang2
                            similarities = futures[index].result()
                            key = f"Cosine_{lang1}_{lang2}"
                            # Extract unique sentence pairs and cosine similarities for the language pair
                            sentences1 = data[lang1]
                            sentences2 = data[lang2]

                            # Create a DataFrame for the language pair
                            df_lang_pair = pd.DataFrame({
                                "LANGUAGE1": [lang1] * len(sentences1),
                                "SENTENCE1": sentences1,
                                "LANGUAGE2": [lang2] * len(sentences2),
                                "SENTENCE2": sentences2,
                                "COSINE SIMILARITY": similarities
                            })

                            # Append the DataFrame to df_pairs
                            df_pairs = pd.concat([df_pairs, df_lang_pair], ignore_index=True)
                            index += 1

            # Write the DataFrame to an Excel file
            output_excel_path = f"/content/drive/MyDrive/{subdirectory}_similarity.xlsx"
            df_pairs.to_excel(output_excel_path, index=False)

if __name__ == "__main__":
    cProfile.run('main()', sort='time')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead

         7239603 function calls (7217866 primitive calls) in 384.781 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2190  333.761    0.152  333.761    0.152 {built-in method torch._C._nn.linear}
      720   18.953    0.026   18.953    0.026 {built-in method torch.matmul}
      360    7.941    0.022    7.941    0.022 {built-in method torch._C._nn.gelu}
      360    5.086    0.014    5.086    0.014 {method 'softmax' of 'torch._C.TensorBase' objects}
      360    4.845    0.013  114.184    0.317 modeling_bert.py:251(forward)
      750    2.822    0.004    2.822    0.004 {built-in method torch.layer_norm}
      328    1.601    0.005    1.601    0.005 {method 'acquire' of '_thread.lock' objects}
      360    1.035    0.003    1.035    0.003 {method 'contiguous' of 'torch._C.TensorBase' objects}
      360    0.970    0.003   30.268    0.084 modeling_bert.py:360(forward)
      360    0.947    0.003  111.844    0.311 modeling


**ESPERIMENTO**



In [3]:
!pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 13.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [4]:
import os
import re
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch

# Download dei dati necessari per NLTK
nltk.download('punkt')

# Inizializzazione del tokenizer e del modello BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Definizione della regex per estrarre la lingua dal nome del file
lang_regex = re.compile(r'_([a-zA-Z]{2})\.txt$')

# Definizione della cartella principale
main_directory = '/content/drive/MyDrive/Wiki download/Puliti_GastronomiaE_en_it_es_de_fr'

# Funzione per suddividere i testi in frasi e calcolare gli embeddings BERT
def process_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        sentences = sent_tokenize(text)
        tokenized_sentences = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in sentences]
        max_length = max(len(sentence) for sentence in tokenized_sentences)
        padded_sentences = [sentence + [0] * (max_length - len(sentence)) for sentence in tokenized_sentences]
        input_ids = torch.tensor(padded_sentences)
        with torch.no_grad():
            outputs = model(input_ids)
            embeddings = outputs.last_hidden_state[:, 0, :]  # Ottieni il vettore di embedding dell'ultimo layer CLS
    return sentences, embeddings

# Funzione per calcolare la similarità coseno tra le frasi
def calculate_cosine_similarity(embeddings):
    similarities = cosine_similarity(embeddings)
    np.fill_diagonal(similarities, -1)  # Evita di calcolare la similarità tra la stessa frase
    return similarities

# Iterazione attraverso le sottocartelle
for subdir, dirs, files in os.walk(main_directory):
    for file in files:
        if file.endswith('.txt'):
            lang_match = lang_regex.search(file)
            if lang_match:
                lang = lang_match.group(1)
                file_path = os.path.join(subdir, file)
                sentences, embeddings = process_text(file_path)
                similarities = calculate_cosine_similarity(embeddings)
                # Creazione del DataFrame per il file Excel
                df = pd.DataFrame(columns=['Language 1', 'Sentence 1', 'Language 2', 'Sentence 2', 'Cosine similarity'])
                for i in range(len(sentences)):
                    for j in range(i + 1, len(sentences)):
                        similarity = similarities[i][j]
                        df = df.append({'Language 1': lang, 'Sentence 1': sentences[i], 'Language 2': lang, 'Sentence 2': sentences[j], 'Cosine similarity': similarity}, ignore_index=True)
                # Salvataggio del DataFrame in un file Excel
                excel_file_path = os.path.join(subdir, f'{lang}_similarity.xlsx')
                df.to_excel(excel_file_path, index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


AttributeError: 'DataFrame' object has no attribute 'append'